# Valve Model

The `Valve` class represents a controllable valve in a water distribution system. Its primary function is to regulate flow based on an opening percentage. The flow rate is calculated using a modified orifice equation, which considers the head difference across the valve. This notebook provides an overview of the `Valve` model and a simulation of its operation.

## State Variables

The state of the valve is described by:

- `opening` (float): The current opening of the valve, as a percentage (0-100).
- `outflow` (float): The calculated flow rate through the valve (m^3/s).

## Parameters

The physical properties of the valve are defined by:

- `discharge_coefficient` (float): The maximum discharge coefficient (when the valve is 100% open).
- `diameter` (float): The diameter of the valve's pipe (m).

## Simulation Example

The following example simulates a valve under varying control signals. The valve's opening is adjusted over time, and the resulting outflow is calculated based on a constant head difference. The simulation results, including the opening percentage and outflow, are then plotted.

**Note:** To run this notebook, ensure you have `matplotlib` installed (`pip install matplotlib`) and that you are running the Jupyter server from the root directory of the project.

In [ ]:
import matplotlib.pyplot as plt
from swp.simulation_identification.physical_objects.valve import Valve
from swp.core.interfaces import State, Parameters

# Initial state and parameters for the valve
initial_state = State(opening=0.0, outflow=0.0)
parameters = Parameters(
    discharge_coefficient=0.85,
    diameter=0.3 # 300mm
)

# Create a Valve instance
valve = Valve(name="control_valve", initial_state=initial_state, parameters=parameters)

# Simulation settings
dt = 1  # time step in seconds
simulation_duration = 120
num_steps = int(simulation_duration / dt)

# Store results for plotting
history = []

# Define external conditions
upstream_head = 20.0
downstream_head = 15.0

# Run the simulation loop
for t in range(num_steps):
    # Vary the valve opening over time
    if t < 30:
        control_signal = 75 # Open to 75%
    elif t > 90:
        control_signal = 25 # Close to 25%
    else:
        control_signal = 100 # Fully open

    action = {
        'control_signal': control_signal,
        'upstream_head': upstream_head,
        'downstream_head': downstream_head
    }
    
    current_state = valve.step(action=action, dt=dt)
    history.append(current_state.copy())

# Print the final state
print("Final State:", valve.get_state())

# Prepare data for plotting
time_steps = [i * dt for i in range(num_steps)]
openings = [s['opening'] for s in history]
outflows = [s['outflow'] for s in history]

# Plot the results
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 8), sharex=True)

ax1.plot(time_steps, openings, label='Valve Opening', drawstyle='steps-post')
ax1.set_ylabel('Opening (%)')
ax1.set_title('Valve Operation Simulation')
ax1.grid(True)
ax1.legend()

ax2.plot(time_steps, outflows, label='Outflow', color='orange')
ax2.set_xlabel('Time (seconds)')
ax2.set_ylabel('Outflow (m^3/s)')
ax2.grid(True)
ax2.legend()

plt.tight_layout()
plt.show()
